In [1]:
import kotlin.io.path.Path
import kotlin.io.path.readText

val input = Path("day5.txt").readText()

val exampleInput = """
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
"""

data class ConversionMap(val conversions: List<Conversion>)
data class Conversion(val destinationRangeStart: Long, val sourceRangeStart: Long, val rangeLength: Int)

val newlinePattern = "(\r)?\n"

fun getBlocks(value: String): List<List<String>> {
    return value.split(Regex(newlinePattern + newlinePattern))
        .map { it.split(Regex(newlinePattern)).filter { it.isNotBlank() } }
}

fun parseSeeds(value: String): List<Long> {
    val (firstBlock) = getBlocks(value)
    return firstBlock[0].substringAfter("seeds: ").split(" ").map { it.toLong() }
}

fun parseConversionMaps(value: String): List<ConversionMap> {
    val blocks = getBlocks(value)
    return blocks.drop(1).map {
        it.drop(1)
            .map {
                val (destinationStart, sourceStart, rangeLength) = it.split(" ").mapNotNull { it.trim().toLongOrNull() }
                Conversion(destinationStart, sourceStart, rangeLength.toInt())
            }
            .let { ConversionMap(it) }
    }
}

val exampleSeeds = parseSeeds(exampleInput)
val exampleConversionMaps = parseConversionMaps(exampleInput)
val seeds = parseSeeds(input)
val conversionMaps = parseConversionMaps(input)

# Part 1

In [2]:
fun getResult(seeds: List<Long>, conversionMaps: List<ConversionMap>): Long {
    return seeds.minOf { getSeedLocation(conversionMaps, it) }
}

fun getSeedLocation(conversionMaps: List<ConversionMap>, seed: Long): Long {
    return conversionMaps.fold(seed) { current, map -> map.convert(current) }
}

fun ConversionMap.convert(value: Long): Long {
    val conversion = conversions.find { it.sourceRange().contains(value) }
    return when (conversion) {
        null -> value
        else -> conversion.convert(value)
    }
}

fun Conversion.convert(value: Long) = destinationRangeStart + value - sourceRangeStart
fun Conversion.sourceRange() = sourceRangeStart..sourceRangeStart + rangeLength - 1


## Example

In [3]:
getResult(exampleSeeds, exampleConversionMaps)

35

## Solution

In [4]:
getResult(seeds, conversionMaps)

389056265

# Part 2

In [5]:
fun parseSeedRanges(value: String): List<LongRange> {
    val (firstBlock) = getBlocks(value)
    val seeds = firstBlock[0].substringAfter("seeds: ").split(" ").map { it.toLong() }
    return seeds.chunked(2).map { (start, length) -> start..start + length - 1 }
}

val exampleSeedRanges = parseSeedRanges(exampleInput)
val seedRanges = parseSeedRanges(input)

fun getResult(seedRanges: List<LongRange>, conversionMaps: List<ConversionMap>): Long {
    val locationRange = 0..999999999999L
    return locationRange.find { 
        val seed = getSeedForLocation(conversionMaps, it)
        seedRanges.any { seed in it }
    }!!
}

fun getSeedForLocation(conversionMaps: List<ConversionMap>, location: Long): Long {
    return conversionMaps.reversed().fold(location) { current, map -> map.convertReverse(current) }
}

fun ConversionMap.convertReverse(value: Long): Long {
    val conversion = conversions.find { it.destinationRange().contains(value) }
    return when (conversion) {
        null -> value
        else -> conversion.convertReverse(value)
    }
}

fun Conversion.destinationRange() = destinationRangeStart..destinationRangeStart + rangeLength - 1
fun Conversion.convertReverse(value: Long) = sourceRangeStart + value - destinationRangeStart

## Example

In [6]:
getResult(exampleSeedRanges, exampleConversionMaps)

46

In [7]:
getResult(seedRanges, conversionMaps)

137516820